<center>

# $\textbf{Migration}$

<center>

### $\textbf{Code}$

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, lit, array, struct

In [22]:
spark = SparkSession.builder.appName('Migration').master("local").enableHiveSupport().getOrCreate()
spark

In [23]:
# Creating dataframe from the csv file and infering the schema
df = spark.read.load("Files/Migration2.csv", format="csv", sep=",", inferschema="true", header="true")

In [24]:
# Drop the columns you don't want
df = df.drop("Series Name", "Series Code", "Country Code")

In [25]:
# Rename the column
df = df.withColumnRenamed("Country Name","country")
df = df.withColumnRenamed("2000 [YR2000]","2000")
df = df.withColumnRenamed("2001 [YR2001]","2001")
df = df.withColumnRenamed("2002 [YR2002]","2002")
df = df.withColumnRenamed("2003 [YR2003]","2003")
df = df.withColumnRenamed("2004 [YR2004]","2004")
df = df.withColumnRenamed("2008 [YR2008]","2008")
df = df.withColumnRenamed("2009 [YR2009]","2009")
df = df.withColumnRenamed("2010 [YR2010]","2010")
df = df.withColumnRenamed("2011 [YR2011]","2011")
df = df.withColumnRenamed("2012 [YR2012]","2012")
df = df.withColumnRenamed("2013 [YR2013]","2013")
df = df.withColumnRenamed("2014 [YR2014]","2014")
df = df.withColumnRenamed("2015 [YR2015]","2015")
df = df.withColumnRenamed("2016 [YR2016]","2016")
df = df.withColumnRenamed("2017 [YR2017]","2017")
df = df.withColumnRenamed("2018 [YR2018]","2018")
df = df.withColumnRenamed("2019 [YR2019]","2019")
df = df.withColumnRenamed("2020 [YR2020]","2020")
df = df.withColumnRenamed("2021 [YR2021]","2021")
df = df.withColumnRenamed("2022 [YR2022]","2022")
df = df.withColumnRenamed("2023 [YR2023]","2023")
df = df.withColumnRenamed("2024 [YR2024]","2024")
df = df.withColumnRenamed("2025 [YR2025]","2025")

In [26]:
# Assuming df is properly defined DataFrame
df = df.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("migration")) 
    for year in list(range(2000, 2005)) + list(range(2008, 2025))
])).alias("data")).selectExpr("country", "data.year", "data.migration")

In [27]:
# Cast columns to their desired types
df = df.withColumn("country", col("country").cast("string"))
df = df.withColumn("year", col("year").cast("int"))
df = df.withColumn("migration", col("migration").cast("int"))

In [28]:
# Filter data for years greater than 2010 and lower than 2024
df = df.filter(df["year"] > 2010)
df = df.filter(df["year"] < 2024)

In [29]:
# Order by country and then by year
df = df.orderBy("country", "year")

In [30]:
#Storing this dataframe in parquet
df.write.mode("overwrite").parquet("FilesParquet/Migration.parquet" )
spark.read.parquet("FilesParquet/Migration.parquet").show()
spark.stop()

+--------------------+----+---------+
|             country|year|migration|
+--------------------+----+---------+
|         Afghanistan|2011|   418796|
|         Afghanistan|2012|   105905|
|         Afghanistan|2013|    48076|
|         Afghanistan|2014|   255611|
|         Afghanistan|2015|  -281739|
|         Afghanistan|2016|   -90238|
|         Afghanistan|2017|   -47090|
|         Afghanistan|2018|   -47205|
|         Afghanistan|2019|    -8082|
|         Afghanistan|2020|   166821|
|         Afghanistan|2021|  -183672|
|         Afghanistan|2022|   -65846|
|         Afghanistan|2023|   -65846|
|Africa Eastern an...|2011|  -683611|
|Africa Eastern an...|2012|  -285231|
|Africa Eastern an...|2013|   -92842|
|Africa Eastern an...|2014|  -199608|
|Africa Eastern an...|2015|   394925|
|Africa Eastern an...|2016|  -864708|
|Africa Eastern an...|2017|  -343075|
+--------------------+----+---------+
only showing top 20 rows

